<a href="https://colab.research.google.com/github/rio1004666/Colab/blob/main/Ch3.%EB%94%A5%EB%9F%AC%EB%8B%9D/9_RNN_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
날짜; 2021/10/28
이름: 강병화
내용: 딥러닝 RNN 실습하기
"""
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Flatten, Dense

In [ ]:
# 시계열 데이터 생성 
time = np.arange(365) # 0~ 364까지 
month_time = (time % 30) / 30
time_series = 10 * np.where(month_time < 0.5, 
              np.cos(2*np.pi*month_time), 
              np.cos(2*np.pi*month_time) + np.random.random(365))
time_series

# 시각화 
plt.figure(figsize=(10, 5))
plt.xlabel('time')
plt.ylabel('value')
plt.plot(np.arange(0, 365),time_series[ : 365])

In [ ]:
# 데이터 정규화 
def make_sequence(time_series, n):
  train_data, train_label = [], []
  for i in range(len(time_series)):
    t = time_series[i:(i+n)]
    if (i+n) < len(time_series):
      train_data.append(t)
      train_label.append(time_series[i+n])
    else:
      break
  return np.array(train_data), np.array(train_label)
train_data,train_label = make_sequence(time_series, 10)
train_label

In [ ]:
# 데이터 셋 구조 변환 (데이터에대한 이해가 필요하다)
train_data = train_data.reshape(-1,10,1) # -1은 행이고 자동으로 정해줌 10은 컬럼갯수 1은 z 차수 
train_label = train_label.reshape(-1,1)
print(train_data[0])

In [ ]:
# 학습데이터 , 라벨 
_train_data = train_data[:300] # 300개까지는 train데이터
_train_label = train_label[:300]
_test_data = train_data[300:]  # 나머지 50여개는 test데이터 
_test_label = train_label[300:]

In [ ]:
# 모델 설계
model = Sequential()
model.add(SimpleRNN(units=32,activation='tanh',input_shape=(10,1))) #u units는 노드갯수 activation은 sigmoid/tanh/relu 다양하게 해봄 
model.add(Dense(1,activation='linear'))
model.summary()

In [ ]:
# 모델설정 
model.compile(optimizer='Adam', loss='mse', metrics=['acc']) #평균제곱오차함수 = mse     정확도확인metrics=['acc']

In [ ]:
# 학습하기
model.fit(_train_data,_train_label,epochs=100) # epochs는 학습횟수 
# 정확도가 0이된다. 왜? 랜덤수라서 맞지 않는다. (추세를 예측하는 것 <= 기존의 데이터를 학습해서  )


In [ ]:
# 성능 평가 
model.evaluate(_test_data,_test_label)

In [ ]:
# 그래프 출력 
result = model.predict(_test_data)
result_range = np.arange(len(_train_label), len(_train_label)+len(result))
result_range

plt.figure(figsize=(10,5))
plt.xlabel('time')
plt.ylabel('value')
plt.plot(result_range,_test_label, color='orange',label='label')
plt.plot(result_range, result, color='blue',label='result')
plt.show()